### Logistic differential equation

In [ ]:
def population_next_generation(growth_rate: float, population: float) -> float:
    r: float = growth_rate
    x: float = population
    return r * x * (1 - x)

def population_nth_generation(growth_rate: float, population_history: list, generations: int) -> list:
    r: float = growth_rate
    n: int = generations
    X: list = population_history
    x0: float = population_history[-1]

    x1 = population_next_generation(r, x0)
    X = X + [x1]
    n = n - 1
    
    if n > 0:
        return population_nth_generation(r, X, n)
    else:
        return X

### Helper functions

In [ ]:
import matplotlib.pyplot as plt

def plot_population_over_time(population_history: list):
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1) 

    x = range(len(population_history))
    y = population_history
        
    ax.cla()
    ax.plot(x, y)

### Next generation population

In [ ]:
r = 2.5
x = 0.5

In [ ]:
x = population_next_generation(r, x)
x

### Calculate all generation's population

In [ ]:
r = 3.0
x = 0.5
n = 100

X = population_nth_generation(r, [x], n)
plot_population_over_time(X)

### Convergence value

In [ ]:
import numpy

def convergence_value(series: list) -> float:
    derivative_first = list( numpy.gradient( series ))
    derivative_second = list( numpy.gradient( series ))

    value_nth = series[-1]
    angle_nth = derivative_first[-1]
    curvature_nth = derivative_second[-1]

    if is_zero(angle_nth) and is_zero(curvature_nth):
        return value_nth
    else: 
        return None

def is_zero(value):
    limit = 5e-5
    return limit > abs(value)
    

In [ ]:
r = 1.0
x = 0.5
n = 150
X = population_nth_generation(r, [x], n)

convergence_value(X)